In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pandas as pd
from pomocnicze_funkcje import readable_time, randomized_service_time


#Naprawa importów pomocniczych funkcji po edycji ich kodu
import importlib
import pomocnicze_funkcje
importlib.reload(pomocnicze_funkcje)

<module 'pomocnicze_funkcje' from 'c:\\Users\\lukas\\OneDrive\\Pulpit\\projekty\\Symulacja_Przychodni\\main\\pomocnicze_funkcje.py'>

In [274]:
class Pacjent:
    def __init__(self, id):
        self.day = 1
        self.id = id
        self.room = None
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None

In [275]:
class Gabinet:
    def __init__(self, id):
        self.id = id
        self.patients_served = 0

In [276]:
class Clinic:
    def __init__(self, number_of_rooms, planned_service_time, queue_array=[], lambda_per_hour=6/60, seed=None, sim_time=120):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.planned_service_time = planned_service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []

In [277]:
def time_between_new_patients(self):
    if self.seed:
        np.random.seed(self.seed)
    return np.random.exponential(1/self.lambda_per_hour)
Clinic.time_between_new_patients = time_between_new_patients

In [278]:
def generate_patients(self):
    while True:
        if self.env.now >= self.sim_time-self.planned_service_time: # Pacjenci nie przychodzą {service_time} przed zamknięciem
            break
        patient = Pacjent(id=self.curr_patient_id)
        patient.arrival_time = self.env.now
        print(f"Czas {readable_time(self.env.now)}: Pacjent {patient.id} przybył do kliniki")
        self.env.process(self.serve_patient(patient))
        self.curr_patient_id += 1
        yield self.env.timeout(self.time_between_new_patients())
Clinic.generate_patients = generate_patients

In [279]:
def serve_patient(self, patient): 
    room = yield self.rooms.get()
    self.queue_array.append(patient.id)
    real_service_time = randomized_service_time(mean=self.planned_service_time, std=2, minimal_time=5)
    if self.env.now >= self.sim_time-self.planned_service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
        print(f"Czas {readable_time(self.env.now)}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
        yield self.rooms.put(room)
        return
    patient.service_start_time = self.env.now
    print(f"Czas {readable_time(self.env.now)}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
    self.queue_array.remove(patient.id)
    yield self.env.timeout(real_service_time)

    patient.service_end_time = self.env.now
    room.patients_served += 1
    print(f"Czas {readable_time(self.env.now)}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {readable_time(patient.arrival_time)} do {readable_time(patient.service_start_time)}")
    patient.room = room.id
    self.processed_patients.append(patient)
    yield self.rooms.put(room)
Clinic.serve_patient = serve_patient

In [280]:
def run(self, day=1):
    print(self.env.now)
    self.env.process(self.generate_patients())
    self.env.run(until=self.sim_time)
    df = pd.DataFrame([
    {
        "day": day,
        "id": p.id,
        "room": p.room,
        "arrival_time": p.arrival_time,
        "service_start_time": p.service_start_time,
        "service_end_time": p.service_end_time,
        "waiting_time": p.service_start_time - p.arrival_time
    }
    for p in self.processed_patients
    ])
    return df.set_index(["day", "id"])
Clinic.run = run

In [281]:
def reset(self):
    self.env = simpy.Environment()
    self.curr_patient_id = 1
    self.list_rooms = [Gabinet(id=i + 1) for i in range(len(self.list_rooms))]
    self.rooms = simpy.Store(self.env, capacity=len(self.list_rooms))
    for room in self.list_rooms:
        self.rooms.put(room)
    self.processed_patients = []
Clinic.reset = reset

In [282]:
def run_multiple_times(self, n):
    multiple_df = pd.DataFrame()
    for i in range(1, n+1):
        print(f"-----------{i}-----------")
        self.reset()
        single_df = self.run(i)
        multiple_df = pd.concat([multiple_df, single_df])
    return multiple_df
Clinic.run_multiple_times = run_multiple_times

In [283]:
clinic = Clinic(number_of_rooms=3, planned_service_time=15, sim_time=480, lambda_per_hour=14/60)

In [ ]:
# single_df = clinic.run(day=1)
df = clinic.run_multiple_times(n=1000) 

df.to_csv("../wyniki/wyniki_bez_um.csv")

In [285]:
df.head()

room  arrival_time  service_start_time  service_end_time  waiting_time
day id                                                                        
1   1      1      0.000000            0.000000         16.331772           0.0
    2      2      6.059012            6.059012         17.866524           0.0
    3      3      7.208505            7.208505         24.406758           0.0
    4      1     21.125465           21.125465         37.080374           0.0
    5      2     24.210372           24.210372         39.689333           0.0